# Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import numpy as np
import torch.optim as optim
import torch.nn as nn
from sklearn.model_selection import ParameterGrid
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import StepLR
import datasets
from weighted_random_search import wrs
from tqdm import tqdm
import matplotlib.pyplot as plt

c:\Users\mikol\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from CNNpy import CNN_3_class, PretrainedAlexNet
from hyperparameter_search import Net_wrapper, GridSearch, RandomSearch 
from datasets import cifar_train_AN , cifar_val_AN

### Sample for testing GridSearch

In [9]:
train_dataset = datasets.cifar_train
val_dataset = datasets.cifar_val

In [10]:
from torch.utils.data import DataLoader, SubsetRandomSampler, Subset

subset_indices = list(range(500))
subset_sampler = SubsetRandomSampler(subset_indices)

subset_train_dataset = Subset(train_dataset, subset_indices)
subset_val_dataset = Subset(val_dataset, subset_indices)

### GridSearch / RandomSearch on AlexNet

In [21]:
test_hyper_params = {'learning_rate': [0.0001, 0.0005, 0.001, 0.005], 'batch_size': [8, 16, 32, 64]}#, 'kernel_size1': [3, 4, 8]}
# my_net = Net_wrapper(model = PretrainedAlexNet, optimizer=optim.Adam, max_epochs=5, learning_rate=0.0005, batch_size=8, 
#                      number_of_filters0=32, number_of_filters1=256, no_neurons=500)
my_net = Net_wrapper(model = PretrainedAlexNet, optimizer=optim.Adam, max_epochs=5)
gs = GridSearch(net=my_net, param_grid=test_hyper_params, step_by_step=True, verbose=1)
# gs = gs.fit(subset_train_dataset, subset_val_dataset)
gs = gs.fit(cifar_train_AN, cifar_val_AN)

c:\Users\mikol\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\mikol\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [13]:
print(gs.best_score)
print(gs.best_params)

0.3333333333333333
{'learning_rate': 0.0005, 'batch_size': 16}


In [14]:
test_hyper_params = {'learning_rate': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1], 'batch_size': [8, 16, 32, 64, 128]}
rs = RandomSearch(my_net, test_hyper_params, verbose=1)
rs.fit(subset_train_dataset, subset_val_dataset, n_trials = 5)

Parameter set: {'learning_rate': 0.0003195254015709299, 'batch_size': 13}
val_accuracy: 0.3353
Parameter set: {'learning_rate': 0.0002668088018810296, 'batch_size': 12}
val_accuracy: 0.3552
Parameter set: {'learning_rate': 0.0002743979608568015, 'batch_size': 13}
val_accuracy: 0.3478
Parameter set: {'learning_rate': 0.0003203191610298302, 'batch_size': 9}
val_accuracy: 0.3448
Parameter set: {'learning_rate': 0.0004868119356054707, 'batch_size': 15}
val_accuracy: 0.3510


In [15]:
print(rs.best_score)
print(rs.best_params)

0.35515872903522994
{'learning_rate': 0.0002668088018810296, 'batch_size': 12}


### Weighted random search test

In [28]:
my_net = Net_wrapper()
param_grid = {'lr': [0.01, 0.05, 0.1, 0.5, 1, 2], 'batch_size': [16, 32, 64, 128, 256]}
w = WeightedRandomSearch(net=my_net, param_grid=test_hyper_params)
N, N_0 = 10, 3
w.fit(subset_train_dataset, subset_val_dataset, N, N_0)
print(w.best_params)

{'learning_rate': 0.01, 'batch_size': 64, 'no_neurons': 64} 0.11944110598415136
{'learning_rate': 0.005, 'batch_size': 64, 'no_neurons': 64} 0.28305288404226303
{'learning_rate': 0.005, 'batch_size': 64, 'no_neurons': 64} 0.28305288404226303
{'learning_rate': 0.005, 'batch_size': 64, 'no_neurons': 64} 0.28305288404226303
{'learning_rate': 0.005, 'batch_size': 64, 'no_neurons': 64} 0.28305288404226303
{'learning_rate': 0.005, 'batch_size': 64, 'no_neurons': 64} 0.28305288404226303
{'learning_rate': 0.005, 'batch_size': 64, 'no_neurons': 64} 0.28305288404226303
{'learning_rate': 0.005, 'batch_size': 64, 'no_neurons': 64} 0.28305288404226303
{'learning_rate': 0.005, 'batch_size': 64, 'no_neurons': 64} 0.28305288404226303
{'learning_rate': 0.005, 'batch_size': 64, 'no_neurons': 64} 0.28305288404226303
{'learning_rate': 0.005, 'batch_size': 64, 'no_neurons': 64} 0.28305288404226303


# Hyperparameter Tuning on CNN3 model

## CNN_3_class on original dataset (without augmentation)

1. Learning rate + batch size

In [16]:
my_net = Net_wrapper(model=CNN_3_class, max_epochs=5)
param_grid = {'learning_rate': [0.0001, 0.0005, 0.001, 0.005, 0.01], 'batch_size': [8, 16, 32, 64, 128]}
gs = GridSearch(net=my_net, param_grid=param_grid, step_by_step=True, verbose=1)
gs = gs.fit(train_dataset, val_dataset)

Current parameter: learning_rate: 0.0001   val_accuracy: 0.5830
Best parameters till now:{'learning_rate': 0.0001}
Current parameter: learning_rate: 0.0005   val_accuracy: 0.6887
Best parameters till now:{'learning_rate': 0.0005}
Current parameter: learning_rate: 0.001   val_accuracy: 0.6761
Best parameters till now:{'learning_rate': 0.0005}
Current parameter: learning_rate: 0.005   val_accuracy: 0.5991
Best parameters till now:{'learning_rate': 0.0005}
Current parameter: learning_rate: 0.01   val_accuracy: 0.1000
Best parameters till now:{'learning_rate': 0.0005}
0.0005
Current parameter: batch_size: 8   val_accuracy: 0.6954
Best parameters till now:{'learning_rate': 0.0005, 'batch_size': 8}
Current parameter: batch_size: 16   val_accuracy: 0.6823
Best parameters till now:{'learning_rate': 0.0005, 'batch_size': 8}
Current parameter: batch_size: 32   val_accuracy: 0.6785
Best parameters till now:{'learning_rate': 0.0005, 'batch_size': 8}
Current parameter: batch_size: 64   val_accuracy

2. Dropout, weight_decay

In [20]:
my_net = Net_wrapper(model=CNN_3_class, max_epochs=5, learning_rate=0.0005, batch_size=8)
param_grid = {'dr': [nn.Dropout(0), nn.Dropout(0.1), nn.Dropout(0.25), nn.Dropout(0.4), nn.Dropout(0.5)], 'weight_decay': [0, 1e-5, 1e-4, 1e-3]}
gs = GridSearch(net=my_net, param_grid=param_grid, step_by_step=True, verbose=1)
gs = gs.fit(train_dataset, val_dataset)

Current parameter: dr: Dropout(p=0, inplace=False)   val_accuracy: 0.7061
Best parameters till now:{'dr': Dropout(p=0, inplace=False)}
Current parameter: dr: Dropout(p=0.1, inplace=False)   val_accuracy: 0.6770
Best parameters till now:{'dr': Dropout(p=0, inplace=False)}
Current parameter: dr: Dropout(p=0.25, inplace=False)   val_accuracy: 0.6614
Best parameters till now:{'dr': Dropout(p=0, inplace=False)}
Current parameter: dr: Dropout(p=0.4, inplace=False)   val_accuracy: 0.6550
Best parameters till now:{'dr': Dropout(p=0, inplace=False)}
Current parameter: dr: Dropout(p=0.5, inplace=False)   val_accuracy: 0.6351
Best parameters till now:{'dr': Dropout(p=0, inplace=False)}
Current parameter: weight_decay: 0   val_accuracy: 0.7047
Best parameters till now:{'dr': Dropout(p=0, inplace=False), 'weight_decay': 0}
Current parameter: weight_decay: 1e-05   val_accuracy: 0.6925
Best parameters till now:{'dr': Dropout(p=0, inplace=False), 'weight_decay': 0}
Current parameter: weight_decay: 0.0

3. Net architecture (kernel size, number of filters)

In [10]:
my_net = Net_wrapper(model=CNN_3_class, max_epochs=5, learning_rate=0.0005, batch_size=8)
param_grid = {'number_of_filters0': [32, 64, 128, 256], 'number_of_filters1': [32, 64, 128, 256]}
gs = GridSearch(net=my_net, param_grid=param_grid, step_by_step=True, verbose=1)
gs = gs.fit(train_dataset, val_dataset)

Current parameter: number_of_filters0: 32   val_accuracy: 0.7007
Best parameters till now:{'number_of_filters0': 32}
Current parameter: number_of_filters0: 64   val_accuracy: 0.6869
Best parameters till now:{'number_of_filters0': 32}
Current parameter: number_of_filters0: 128   val_accuracy: 0.7005
Best parameters till now:{'number_of_filters0': 32}
Current parameter: number_of_filters0: 256   val_accuracy: 0.6403
Best parameters till now:{'number_of_filters0': 32}
Current parameter: number_of_filters1: 32   val_accuracy: 0.6976
Best parameters till now:{'number_of_filters0': 32, 'number_of_filters1': 32}
Current parameter: number_of_filters1: 64   val_accuracy: 0.7197
Best parameters till now:{'number_of_filters0': 32, 'number_of_filters1': 64}
Current parameter: number_of_filters1: 128   val_accuracy: 0.7115
Best parameters till now:{'number_of_filters0': 32, 'number_of_filters1': 64}
Current parameter: number_of_filters1: 256   val_accuracy: 0.7270
Best parameters till now:{'number_

TypeError: CNN_3_class.__init__() got an unexpected keyword argument 'kernel_size1'

In [19]:
my_net = Net_wrapper(model=CNN_3_class, max_epochs=5, learning_rate=0.0005, batch_size=8, 
                     number_of_filters0=32, number_of_filters1=256)
param_grid = {'kernel_size1': [3,4,5], 'kernel_size2':[3,4,5], 'no_neurons': [32, 64, 128, 256]}
gs = GridSearch(net=my_net, param_grid=param_grid, step_by_step=True, verbose=1)
gs = gs.fit(train_dataset, val_dataset)

4. Num of epochs (run a lot of epochs and stop when validation accuracy starts to decrease)

In [43]:
my_net = Net_wrapper(model=CNN_3_class, max_epochs=20, learning_rate=0.0005, batch_size=8, verbose=1)
my_net.score(train_dataset, val_dataset)

Epoch 1/20


100%|██████████| 5000/5000 [02:22<00:00, 35.16it/s]


train_loss:1.367179244017601, train_acc: 0.5074


100%|██████████| 1250/1250 [00:19<00:00, 62.74it/s]


val_loss:1.1311261782169342, val_acc: 0.6056
Epoch 2/20


...

train_loss:0.08646666240554222, train_acc: 0.9746


100%|██████████| 1250/1250 [00:17<00:00, 71.20it/s]


val_loss:1.7931744132095948, val_acc: 0.6888
Epoch 16/20


  7%|▋         | 356/5000 [00:11<02:25, 31.96it/s]


KeyboardInterrupt: 